Program to check if a photo is of a national park, and whether the photo is of a bird.

### 1. Download images of birds and non-birds



In [1]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')